<a href="https://colab.research.google.com/github/joshdaniel/water/blob/main/water/datasets/OpenLandMap_SoilType/usda_soil_type_available_water_capacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## USDA Soil Type : Available Water Capacity

Compute water holding capacity of soil at 6 depths just from soil type.
Author: Josh Daniel


In [4]:
# This may need to be run anytime there's a new runtime.
!pip install earthshot

     |████████████████████████████████| 10.1MB 4.1MB/s 
     |████████████████████████████████| 256kB 34.0MB/s 
     |████████████████████████████████| 20.1MB 1.2MB/s 
     |████████████████████████████████| 8.9MB 29.3MB/s 
     |████████████████████████████████| 153kB 43.3MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 2.2MB 44.7MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
  Created wheel for bokeh: filename=bokeh-2.2.3-cp37-none-any.whl size=9296311 sha256=29f05c7e5169ce140c38d695f6005e6f0829598adac6b26d87332d017eddbac3
  Stored in directory: /root/.cache/pip/wheels/dc/96/7b/9c9d48ed392511bc708e39580e18dc7a92f475795cd26b51bf
  Created wheel for earthengine-api: filename=earthengine_api-0.1.243-cp37-none-any.whl size=177524 sha256=0137013ff157c278389774523f8a5ef6b676f47593a03734a2674ce92cd5bd96
  Stored in directory: /root/.cache/pip/wheels/42/5c/8d/2e172f7ff26266

In [1]:
import ee

from earthshot import mon_stats
from earthshot import water_common
from earthshot import water_viz as vis

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=yJmFRqcThj3-1hYtPRRxU5MaIdOtjwrolIA5RENV8RI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7i7uvDcZAin94ApMAL-mygRyYZNDfMDU3s5yoqLMhx95Ps0iCOYS0

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
dataset = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

```
OpenLandMap Soil texture class (USDA system)
Resolution: 250m

Bands:
b0 Soil texture class (USDA system) at 0 cm depth
b10 Soil texture class (USDA system) at 10 cm depth
b30 Soil texture class (USDA system) at 30 cm depth
b60 Soil texture class (USDA system) at 60 cm depth
b100 Soil texture class (USDA system) at 100 cm depth
b200 #Soil texture class (USDA system) at 200 cm depth

Values:
Sa = Sand
Si = Silt
Cl = Clay
Lo = Loam
Value Color Description
1 d5c36b Cl
2 b96947 SiCl
3 9d3706 SaCl
4 ae868f ClLo
5 f86714 SiClLo
6 46d143 SaClLo
7 368f20 Lo
8 3e5a14 SiLo
9 ffd557 SaLo
10 fff72e Si
11 ff5a9d LoSa
12 ff005b Sa
```

```
Plant available water holding capacity by soil type.

Available water is expressed as a volume fraction (0.20),
as a percentage (20%), or as an amount (in inches). An
example of a volume fraction is water in inches per inch
of soil. If a soil has an available water fraction of 0.20, a
10 inch zone then contains 2 inches of available water.

see: https://www.nrcs.usda.gov/Internet/FSE_DOCUMENTS/nrcs142p2_051279.pdf
Note: This code uses the middle of each range from that document.

Here's another reference with similar numbers but different units:
https://ucanr.edu/sites/UrbanHort/Water_Use_of_Turfgrass_and_Landscape_Plant_Materials/Soil_Water_Holding_Characteristics/
```



In [5]:
water_holding_fraction_by_soil_type = {
  1: 0.15, # Cl
  2: 0.15, # SiCl
  3: 0.125, # SaCl
  4: 0.125, # ClLo
  5: 0.2, # SiClLo
  6: 0.125, # SaClLo
  7: 0.125, # Lo
  8: 0.2, # SiLo
  9: 0.05, # SaLo
  10: 0.2, # Si
  11: 0.05, # LoSa
  12: 0.05 # Sa
}

```
TODO: Move this to shared code library.

This should generate a string suitable for ee.Image.expression() to map categorical data into
numerical value data on a single Image.

Inputs:
band_name (str): name of the band in the Image.
cat_val_dict (dict): a dictionary that maps from categories to numerical values.

Outputs:
An example expression string may look like the following.
  '(b("b0") == 1) ? 0.5 ' +
  ': (b("b0") == 2) ? 0.75 ' +
  ': 0'
```

In [6]:
def generate_category_to_value_expression(band_name, cat_val_dict):
  ret = ''
  for key, value in cat_val_dict.items():
    ret += '(b("%s") == %s) ? %s : ' % (band_name, key, value)
  ret += '0'
  return ret

In [7]:
category_to_value_expression_b0 = generate_category_to_value_expression('b0', water_holding_fraction_by_soil_type)
category_to_value_expression_b10 = generate_category_to_value_expression('b10', water_holding_fraction_by_soil_type)
category_to_value_expression_b30 = generate_category_to_value_expression('b30', water_holding_fraction_by_soil_type)
category_to_value_expression_b60 = generate_category_to_value_expression('b60', water_holding_fraction_by_soil_type)
category_to_value_expression_b100 = generate_category_to_value_expression('b100', water_holding_fraction_by_soil_type)
category_to_value_expression_b200 = generate_category_to_value_expression('b200', water_holding_fraction_by_soil_type)


In [8]:
# Lookup estimated water holding capacity from soil type for each layer.
plant_available_water_from_soil_type_b0 = dataset.expression(category_to_value_expression_b0)
plant_available_water_from_soil_type_b10 = dataset.expression(category_to_value_expression_b10)
plant_available_water_from_soil_type_b30 = dataset.expression(category_to_value_expression_b30)
plant_available_water_from_soil_type_b60 = dataset.expression(category_to_value_expression_b60)
plant_available_water_from_soil_type_b100 = dataset.expression(category_to_value_expression_b100)
plant_available_water_from_soil_type_b200 = dataset.expression(category_to_value_expression_b200)


In [9]:
visualization_pawc = {
  'min': 0.0,
  'max': 0.2,
  'dimensions': 512,
  'palette': [
    "black", "purple"
  ]
}

In [10]:
# Display a thumbnail
display(vis.Image(url = plant_available_water_from_soil_type_b0.getThumbURL(visualization_pawc)))

In [11]:
# Display all 6 layers
the_map = vis.folium_map(location=[20, 0], zoom_start=3, height=500)
the_map.add_ee_layer(plant_available_water_from_soil_type_b0, visualization_pawc, 'b0 plant available water by soil type')
the_map.add_ee_layer(plant_available_water_from_soil_type_b10, visualization_pawc, 'b10 plant available water by soil type')
the_map.add_ee_layer(plant_available_water_from_soil_type_b30, visualization_pawc, 'b30 plant available water by soil type')
the_map.add_ee_layer(plant_available_water_from_soil_type_b60, visualization_pawc, 'b60 plant available water by soil type')
the_map.add_ee_layer(plant_available_water_from_soil_type_b100, visualization_pawc, 'b100 plant available water by soil type')
the_map.add_ee_layer(plant_available_water_from_soil_type_b200, visualization_pawc, 'b200 plant available water by soil type')
vis.folium_display(the_map)

## Export the USDA soil type 

In [25]:
bbox_dict = water_common.bboxes()
bbox_name = 'world'
bbox = bbox_dict[bbox_name]
collection_name = 'users/joshdanielearthengine/usda_soil_type'
asset_base_name = 'usda_soil_type_available_water_capacity_'
variables = [(plant_available_water_from_soil_type_b0, 'b0'),
             (plant_available_water_from_soil_type_b10, 'b10'),
             (plant_available_water_from_soil_type_b30, 'b30'),
             (plant_available_water_from_soil_type_b60, 'b60'),
             (plant_available_water_from_soil_type_b100, 'b100'),
             (plant_available_water_from_soil_type_b200, 'b200')]
max_pixels = 100000000000  #For global is quite large

for image, band_name in variables:
  description = 'soil_type_plant_available_water_capacity_{}'.format(band_name)
  print (description)
  assetId = '{}/{}'.format(collection_name, description)
  print (assetId)

  xx = ee.batch.Export.image.toAsset(
      image,
      description=description, 
      assetId=assetId,
      region=bbox,
      maxPixels=max_pixels).start()

soil_type_plant_available_water_capacity_b0
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b0
soil_type_plant_available_water_capacity_b10
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b10
soil_type_plant_available_water_capacity_b30
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b30
soil_type_plant_available_water_capacity_b60
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b60
soil_type_plant_available_water_capacity_b100
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b100
soil_type_plant_available_water_capacity_b200
users/joshdanielearthengine/usda_soil_type/soil_type_plant_available_water_capacity_b200
